## Ingesting PDF

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

In [3]:
!pip install git+https://github.com/bigcat88/pillow_heif.git



  Cloning https://github.com/bigcat88/pillow_heif.git to /private/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/pip-req-build-z0c7qrde
  Running command git clone --filter=blob:none --quiet https://github.com/bigcat88/pillow_heif.git /private/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/pip-req-build-z0c7qrde
  Resolved https://github.com/bigcat88/pillow_heif.git to commit c608c10b7258ec7118ebfab8991f2ce2731f59b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pillow_heif (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [74 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-312
      creating build/lib.macosx-10.9-x86_64-cpython-312/pillow_heif
      copying pillow_heif/AvifImagePlugin.py -> build/lib.macosx-1

In [4]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [5]:
local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

ModuleNotFoundError: No module named 'cv2'

In [ ]:

# Preview first page
data[0].page_content

## Vector Embeddings

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!ollama list

In [ ]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

## Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))

In [ ]:
chain.invoke("What are the 5 pillars of global cooperation?")

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()